# Build the Conversational Search Building Blocks

<div>
<img src="Module_1_Build_Conversational_Search/module1/all_components.png", width="800"/>
</div>



### In this lab, We will build the above components one by one to design an end to end conversational search application where you can simply upload a pdf and ask questions over the pdf content. The components include,

* **OpenSearch** as the Vector Database
* **Sagemaker endpoints** to host Embedding and the large language models
* **DynamoDB** as the memory store
* **Lambda functions** as the Document and Query Enoders
* **Ec2 instance** to host the web application

---

The lab includes the following steps:

1. [Get the Cloudformation outputs](#Get-the-Cloudformation-outputs)
2. [Component 1 : OpenSearch Vector DB](#Component-1-:-OpenSearch-Vector-DB)
3. [Component 2 : Embedding and LLM Endpoints](#Component-2-:--Embedding-and-LLM-Endpoints)
4. [Component 3 : Memory Store](#Component-3-:--Memory-Store)
5. [Component 4 : Document and Query Encoder](#Component-4-:--Document-and-Query-Encoder)
6. [Component 5 : Client WebServer](#Component-5-:-Client-WebServer)


## Get the Cloudformation outputs

Here, we retrieve the services that are already deployed as a part of the cloudformation template to reduce the deployemnt time for the purpose of this lab. These services include OpenSearch cluster and the Sagemaker endpoints for the LLM and the embedding models.

We also create a **env_variables** dictionary to store the parameters needed to passed onto Lambda functions (Encoders) as environment variables.

In [ ]:
import sagemaker, boto3, json, time
from sagemaker.session import Session
import subprocess
from IPython.utils import io
from Module_1_Build_Conversational_Search import lambda_URL, lambda_exec_role as createRole, lambda_function as createLambda

cfn = boto3.client('cloudformation')
response = cfn.list_stacks(StackStatusFilter=['CREATE_COMPLETE'])
for cfns in response['StackSummaries']:
    if('semantic-search' in cfns['StackName']):
        stackname = cfns['StackName']

cfn_outputs = cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']

for output in cfn_outputs:
    if('s3' in output['OutputKey'].lower()):
        s3_bucket = output['OutputValue']

aws_region = boto3.Session().region_name        
env_variables = {"aws_region":aws_region}

cfn_outputs

## Component 1 : OpenSearch Vector DB

<div>
<img src="Module_1_Build_Conversational_Search/module1/vectordb.png" width="600"/>
</div>

Here, we retrieve the Endpoint of the OpenSearch cluster from the cloudformation outputs, pass it to the env_variables dictionary and also describe the cluster to see the highlevel configuration quickly.

In [ ]:
for output in cfn_outputs:
    if('opensearch' in output['OutputKey'].lower()):
        env_variables[output['OutputKey']] = output['OutputValue']
        
opensearch_ = boto3.client('opensearch')

response = opensearch_.describe_domain(
    DomainName=env_variables['OpenSearchDomainName']
)

print("OpenSearch Version: "+response['DomainStatus']['EngineVersion']+"\n")
print("OpenSearch Configuration\n------------------------\n")
print(json.dumps(response['DomainStatus']['ClusterConfig'], indent=4))        

## Component 2 : Embedding and LLM Endpoints


<div>
<img src="Module_1_Build_Conversational_Search/module1/ml_models.png" width="600"/>
</div>

Here we retrieve the endpoints of the LLM and the embedding models from the cloudformation outputs, pass it to the env_variables dictionary and also describe the endpoints to see the highlevel configuration quickly


In [ ]:
sagemaker_ = boto3.client('sagemaker')

for output in cfn_outputs:
    if('endpointname' in output['OutputKey'].lower()):
        env_variables[output['OutputKey']] = output['OutputValue']
        print(output['OutputKey'] + " : "+output['OutputValue']+"\n"+"------------------------------------------------")
        print(json.dumps(sagemaker_.describe_endpoint_config(EndpointConfigName = sagemaker_.describe_endpoint(
    EndpointName=output['OutputValue']
                            )['EndpointConfigName'])['ProductionVariants'][0],indent = 4))
                        

## Component 3 : Memory Store

 

<div>
<img src="Module_1_Build_Conversational_Search/module1/memory.png" width="600"/>
</div>

Here we create the Dynamo DB table which is used as the memory store to store the history of conversations happening in the application. SessionId is the unique identifier of a conversation entry in the table which acts as the partition column. 

In [ ]:
dynamo = boto3.client('dynamodb')

response = dynamo.create_table(
    TableName='conversation-history-memory',
    AttributeDefinitions=[
        {'AttributeName': 'SessionId', 'AttributeType': 'S'}
    ],
    KeySchema=[
        { 'AttributeName': 'SessionId', 'KeyType': 'HASH'}
    ],
    ProvisionedThroughput={'ReadCapacityUnits': 5,'WriteCapacityUnits': 5}
)
env_variables['DynamoDBTableName'] = response['TableDescription']['TableName']

print("dynamo DB Table, '"+response['TableDescription']['TableName']+"' is created")

## Component 4 : Document and Query Encoder

<div>
<img src="Module_1_Build_Conversational_Search/module1/encoders.png" width="600"/>
</div>

Here we create the Lambda functions for the document and query encoder. These lambda funcitons are packaged with Langchian module. We perform the following steps,
1. Package the dependant libraries (Langchain) and handler files for lambda functions as zip files and push to S3
2. Create the IAM role with sufficient permissions that can be assumed by the lambda functions
3. Create the Lambda functions in python3.9 by passing the already created env_variables as environment variables for the functions.
```
  { 
    'aws_region': 'us-west-2',
    'OpenSearchDomainEndpoint': 'xxxx',
    'OpenSearchDomainName': 'opensearchservi-xxxxxx',
    'OpenSearchSecret': 'xxxx',
    'EmbeddingEndpointName': 'opensearch-gen-ai-embedding-gpt-j-xx-xxxxx',
    'LLMEndpointName': 'opensearch-gen-ai-llm-falcon-7b-xx-xx',
    'DynamoDBTableName': 'conversation-history-memory'
  }
```

4. Create external Lambda URL for queryEncoder lambda to be called from outside world

In [ ]:
#Get the ARN of the IAM role (deployed in cloud formation) for the lambda to assume.

iam_ = boto3.client('iam')
response = iam_.get_role(
    RoleName='LambdaRoleForEncoders'
)

roleARN = response['Role']['Arn']

#Create Lambda functions
encoders = ['queryEncoder','documentEncoder']
createLambda.createLambdaFunction(encoders,roleARN,env_variables)

#Create Lambda URL
account_id=roleARN.split(':')[4]
query_invoke_URL = lambda_URL.createLambdaURL('queryEncoder',account_id)
print("\nLambdaURL created, URL: "+query_invoke_URL)

## Component 5 : Client WebServer

<div>
<img src="Module_1_Build_Conversational_Search/module1/webserver.png" width="600"/>
</div>

Before you go into the final step, you need to add your current **PUBLIC IP** address to the ec2 security group so that you are able to access the web application (chat interface) that you are going to host in the next step.

<h3 style="color:red;"><U>Warning</U></h3>
<h4>Without doing the below steps, you will not be able to proceed further.</h4>

<div>
    <h3 style="color:red;"><U>Enter your IP address </U></h3>
    <h4> STEP 1. Get your IP address <span style="display:inline;color:blue"><a href = "https://ipinfo.io/ip ">HERE</a></span></h4>
</div>

<h4>STEP 2. Run the below cell </h4>
<h4>STEP 3. Paste the IP address in the input box that prompts you to enter your IP</h4>
<h4>STEP 4. Press ENTER</h4>

In [ ]:
my_ip = (input("Enter your IP : ")).split(".")
my_ip.pop()
IP = ".".join(my_ip)+".0/24"

port_protocol = {443:'HTTPS',80:'HTTP',8501:'streamlit'}

IpPermissions = []

for port in port_protocol.keys():
     IpPermissions.append({
            'FromPort': port,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': IP,
                    'Description': port_protocol[port]+' access',
                },
            ],
            'ToPort': port,
        })

IpPermissions

for output in cfn_outputs:
    if('securitygroupid' in output['OutputKey'].lower()):
        sg_id = output['OutputValue']
        
#sg_id = 'sg-0e0d72baa90696638'

ec2_ = boto3.client('ec2')        

response = ec2_.authorize_security_group_ingress(
    GroupId=sg_id,
    IpPermissions=IpPermissions,
)

print("\nIngress rules added for the security group, ports:protocol - "+json.dumps(port_protocol)+" with my ip - "+IP)

Finally, We are ready to host our conversational search application, here we perform the following steps, Steps 2-5 are achieved by executing the terminal commands in the ec2 instance using a SSM client.
1. Update the web application code files with lambda url (in [api.py](https://github.com/aws-samples/semantic-search-with-amazon-opensearch/blob/main/generative-ai/Module_1_Build_Conversational_Search/webapp/api.py)) and s3 bucket name (in [app.py](https://github.com/aws-samples/semantic-search-with-amazon-opensearch/blob/main/generative-ai/Module_1_Build_Conversational_Search/webapp/app.py))
2. Archieve the application files and push to the configured s3 bucket.
3. Download the application (.zip) from s3 bucket into ec2 instance (/home/ec2-user/), and uncompress it.
4. We install the streamlit and boto3 dependencies inside a virtual environment inside the ec2 instance.
5. Start the streamlit application.

In [ ]:
#modify the code files with lambda url and s3 bucket names
query_invoke_URL_cmd = query_invoke_URL.replace("/","\/")

with io.capture_output() as captured:
    #Update the webapp files to include the s3 bucket name and the LambdaURL
    !sed -i 's/API_URL_TO_BE_REPLACED/{query_invoke_URL_cmd}/g' Module_1_Build_Conversational_Search/webapp/api.py
    !sed -i 's/pdf-repo-uploads/{s3_bucket}/g' Module_1_Build_Conversational_Search/webapp/app.py
    #Push the WebAPP code artefacts to s3
    !cd Module_1_Build_Conversational_Search/webapp && zip -r ../webapp.zip *
    !aws s3 cp Module_1_Build_Conversational_Search/webapp.zip s3://$s3_bucket
        
#Get the Ec2 instance ID which is already deployed
response = cfn.describe_stack_resources(
    StackName=stackname
)
for resource in response['StackResources']:
    if(resource['ResourceType'] == 'AWS::EC2::Instance'):
        ec2_instance_id = resource['PhysicalResourceId']
   
# function to execute commands in ec2 terminal
def execute_commands_on_linux_instances(client, commands):
    resp = client.send_command(
        DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
        Parameters={'commands': commands},
        InstanceIds=[ec2_instance_id],
    )
    return resp['Command']['CommandId']

ssm_client = boto3.client('ssm') 

commands = [
            'aws s3 cp s3://'+s3_bucket+'/webapp.zip /home/ec2-user/',
            'unzip -o /home/ec2-user/webapp.zip -d /home/ec2-user/'  ,  
            'sudo chmod -R 0777 /home/ec2-user/',
            'aws s3 cp /home/ec2-user/pdfs s3://'+s3_bucket+'/sample_pdfs/ --recursive',
            'python3 -m venv /home/ec2-user/.myenv',
            'source /home/ec2-user/.myenv/bin/activate',
            'pip install streamlit',
            'pip install boto3',
    
            #start the web applicaiton
            'streamlit run /home/ec2-user/app.py',
            ]

command_id = execute_commands_on_linux_instances(ssm_client, commands)

ec2_ = boto3.client('ec2')
response = ec2_.describe_instances(
    InstanceIds=[ec2_instance_id]
)
public_ip = response['Reservations'][0]['Instances'][0]['PublicIpAddress']
print("Please wait while the application is being hosted . . .")
time.sleep(10)
print("\nApplication hosted successfully")
print("\nClick the below URL to open the application. It may take up to a minute or two to start the application, Please keep refreshing the page if you are seeing connection error.\n")
print('http://'+public_ip+":8501")
print("\nCheck the below video on how to interact with the application")

<h3>Play with the chat application</h3>
<div>
<img src="Module_1_Build_Conversational_Search/module1/module1.gif"/>
</div>